In [37]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
import plotly.graph_objects as go

In [38]:
alpaca_secret_key = 'rXoCM9s8Zug4baosmoqZ52Quo1HTE4APznEqHRdk'
alpaca_api_key = 'PKQ7AGGJ1RU7HOMRQRKM'

In [39]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [42]:
#Import Returns Data
ETF_Return_Path = Path('ETF_Daily_Returns')
Joe_stock_path = Path('/Users/whitneyturner/Project-FI-Data/Performance_Data/Joe_Stock_Portfolio/Joes_Weighted_Returns')
Joe_fidelity_path = Path('/Users/whitneyturner/Project-FI-Data/Joe_Fidelity_Stock_Data/Resources/Joe_Fidelity_Weighted_Returns')
sharpe_path = Path('/Users/whitneyturner/Project-FI-Data/STD_and_Sharpe_Risk_Analysis/Sharpe_Ratio_returns.csv')
larry_path = Path('/Users/whitneyturner/Project-FI-Data/Performance_Data/larry_port_retun.csv')

In [43]:
#Read in data
etf_df = pd.read_csv(ETF_Return_Path, index_col="Date", parse_dates=True, infer_datetime_format=True)
joe_stock_df = pd.read_csv(Joe_stock_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
joe_fid_df = pd.read_csv(Joe_fidelity_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
larry_df = pd.read_csv(larry_path, index_col = "Date", parse_dates=True, infer_datetime_format=True)

In [44]:
#Combine Daily Returns into one data framerame
combined_df = pd.concat([etf_df, joe_stock_df, joe_fid_df, larry_df], axis = 'columns', join = 'inner').dropna()
combined_df.head()

,ARKF,ARKK,ARKQ,Joe's Stock Returns,Joe's fidelity Returns,Larry's Returns
Date,,,,,,
2020-12-17 16:00:00,0.022276,0.017104,0.020144,0.039053,0.008710,0.019379
2020-12-18 16:00:00,-0.001387,0.009756,0.007490,0.038431,-0.003129,0.011046
2020-12-21 16:00:00,0.000397,0.031736,0.009597,0.021656,0.009080,0.013163
2020-12-22 16:00:00,0.017053,0.028247,0.024903,0.022790,-0.001908,0.009665
2020-12-23 16:00:00,-0.008969,-0.000666,0.009798,-0.002294,0.003121,-0.002409


In [45]:
#Read in Sharpe Ratio Data
sharpe_df = pd.read_csv(sharpe_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
sharpe_df.head()

,SP500,ARKK,BABA,PTON,GM,TACO,AUY,AA,NIO,PFE,XOM,ZNGA
Date,,,,,,,,,,,,
2019-09-27 16:00:00,0.000000,-0.009984,-0.051543,-0.020186,-0.005052,0.007843,-0.029586,-0.018104,-0.107143,0.012094,0.007186,-0.026667
2019-10-01 16:00:00,0.000000,-0.023082,-0.012438,-0.103187,-0.036553,-0.012708,0.028302,-0.010463,-0.153846,-0.014101,-0.023509,-0.006873
2019-10-02 16:00:00,0.000000,-0.015036,0.003754,0.003998,-0.039601,-0.025743,0.024465,-0.028197,0.204545,-0.020560,-0.026106,-0.008651
2019-10-08 16:00:00,-0.051282,-0.033547,-0.037963,-0.012761,-0.025036,-0.001027,0.035503,-0.039481,0.096774,-0.011196,-0.019406,-0.005008
2019-10-09 16:00:00,0.000000,0.014032,0.020132,0.034037,0.007674,-0.019527,-0.017143,-0.004327,-0.035294,0.007449,0.011094,0.028523


In [46]:
# Calculate std dev
all_portfolio_std = sharpe_df.std()
all_portfolio_std

SP500    0.069670
ARKK     0.031003
BABA     0.025135
PTON     0.046693
GM       0.036100
TACO     0.048196
AUY      0.038882
AA       0.050346
NIO      0.062686
PFE      0.021034
XOM      0.031653
ZNGA     0.026231
dtype: float64

In [47]:
### Joe's Simulations Data frame
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes between now and 5 years ago.
start_date = pd.Timestamp("2018-05-24", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-24", tz="America/New_York").isoformat()

# Set the ticker information
tickers = ["GM","TACO","AUY","WORK","AAPL"]

# Get 5 year's worth of historical price data
joes_ticker = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df

# Display sample data
joes_ticker.head()

AAPL                                     AUY  \
                             open    high     low   close    volume  open   
time                                                                        
2018-05-24 00:00:00-04:00  188.77  188.84  186.21  188.21  15019898  2.84   
2018-05-25 00:00:00-04:00  188.23  189.65  187.65  188.58  13702107  2.90   
2018-05-29 00:00:00-04:00  187.60  188.75  186.87  187.86  17042382  2.83   
2018-05-30 00:00:00-04:00  187.72  188.00  186.78  187.50  14392159  2.88   
2018-05-31 00:00:00-04:00  187.25  188.23  186.14  186.88  16629099  2.91   

                                                        ...   TACO         \
                           high     low close   volume  ...   open   high   
time                                                    ...                 
2018-05-24 00:00:00-04:00  2.92  2.8258  2.91  8418694  ...  11.77  11.84   
2018-05-25 00:00:00-04:00  2.94  2.8600  2.86  8681001  ...  11.54  12.17   
2018-05-29 00:00:00-04:00  2.90  2.8100  2.87  6205123  ...  12.05  12.12   
2018-05-30 00:00:00-04:00  2.95  2.8730  2.91  7032656  ...  11.90  12.23   
2018-05-31 00:00:00-04:00  2.94  2.8900  2.92  8482293  ...  12.26  12.43   

                                                  WORK                        
                             low  close    volume open high low close volume  
time                                                                          
2018-05-24 00:00:00-04:00  11.53  11.55  183250.0  NaN  NaN NaN   NaN    NaN  
2018-05-25 00:00:00-04:00  11.54  12.07  409256.0  NaN  NaN NaN   NaN    NaN  
2018-05-29 00:00:00-04:00  11.79  11.89  198195.0  NaN  NaN NaN   NaN    NaN  
2018-05-30 00:00:00-04:00  11.90  12.21  187752.0  NaN  NaN NaN   NaN    NaN  
2018-05-31 00:00:00-04:00  12.00  12.03  149811.0  NaN  NaN NaN   NaN    NaN  

[5 rows x 25 columns]

In [15]:
##Larry' Simulation data frame
# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes between now and 5 years ago.
start_date = pd.Timestamp("2018-05-24", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-24", tz="America/New_York").isoformat()

# Set the ticker information
larrys_tickers = ['AA', 'NIO', 'PFE', 'XOM', 'ZNGA']

# Get 5 year's worth of historical price data
larrys_ticker = api.get_barset(
    larrys_tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df

# Display sample data
larrys_ticker.head()

AA                                 NIO           \
                            open    high    low  close   volume open high low   
time                                                                            
2018-05-24 00:00:00-04:00  48.62  49.440  48.55  49.15  1132704  NaN  NaN NaN   
2018-05-25 00:00:00-04:00  48.74  48.825  47.66  47.92  2154178  NaN  NaN NaN   
2018-05-29 00:00:00-04:00  47.57  47.950  46.82  47.30  1599177  NaN  NaN NaN   
2018-05-30 00:00:00-04:00  48.11  48.660  47.50  48.54  1353344  NaN  NaN NaN   
2018-05-31 00:00:00-04:00  49.98  50.430  47.81  48.06  2899262  NaN  NaN NaN   

                                        ...    XOM                       \
                          close volume  ...   open   high    low  close   
time                                    ...                               
2018-05-24 00:00:00-04:00   NaN    NaN  ...  81.35  81.36  79.95  80.26   
2018-05-25 00:00:00-04:00   NaN    NaN  ...  79.28  79.33  78.09  78.70   
2018-05-29 00:00:00-04:00   NaN    NaN  ...  78.15  78.73  77.53  78.40   
2018-05-30 00:00:00-04:00   NaN    NaN  ...  79.25  81.66  79.22  81.51   
2018-05-31 00:00:00-04:00   NaN    NaN  ...  80.81  81.55  80.69  81.24   

                                     ZNGA                                
                             volume  open   high    low close    volume  
time                                                                     
2018-05-24 00:00:00-04:00   9966548  4.14  4.185  4.070  4.15   7431723  
2018-05-25 00:00:00-04:00  10994921  4.12  4.220  4.110  4.18   6848868  
2018-05-29 00:00:00-04:00  10346262  4.16  4.190  4.130  4.19   5380287  
2018-05-30 00:00:00-04:00  12728594  4.20  4.430  4.200  4.33  24457856  
2018-05-31 00:00:00-04:00   9428701  4.34  4.520  4.331  4.41  23173555  

[5 rows x 25 columns]

In [16]:
# Configure a Monte Carlo simulation to forecast twenty years cumulative returns for Joe's Portfolio
joes_MC_even_dist = MCSimulation(
    portfolio_data = joes_ticker,
    weights = [.2, .2, .2, .2, .2],
    num_simulation = 500,
    num_trading_days = 252*20
)

In [17]:
# Run a Monte Carlo simulation to forecast twenty years cumulative returns
joes_MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.002513,0.999157,0.967831,0.994938,0.995784,0.986172,0.992067,1.022025,1.005320,1.001764,...,0.998048,0.997633,1.014283,0.972331,0.998725,1.001970,1.002999,1.014680,1.001576,0.984796
2,0.996944,0.989742,0.963834,1.015713,0.995687,0.958404,1.002101,1.041096,1.013679,0.999994,...,0.987783,1.006001,1.013840,0.957789,0.988695,0.994837,1.010893,1.037127,0.999428,0.984054
3,0.978861,0.991996,0.972020,1.007947,0.987639,0.957750,1.030229,1.049319,1.020875,1.007534,...,0.992869,0.970920,1.006731,0.970666,0.983848,1.000311,1.022981,1.033112,0.986401,0.959311
4,0.999498,0.988687,0.968500,1.019759,0.991226,0.976470,1.036679,1.070138,1.035059,0.995347,...,0.993440,0.973012,1.042911,0.964104,0.981185,1.001614,1.055962,1.043515,0.998976,0.966978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5036,98.142645,44.610608,207.048708,87.509586,193.659656,10.453944,30.580400,140.602924,28.476148,7.737550,...,354.557040,6.735627,12.290630,27.411984,101.100389,47.484334,20.724883,21.550848,25.410462,105.139026
5037,98.423844,43.303946,206.261702,85.783922,194.898558,10.351446,30.708746,142.041313,28.165272,7.758220,...,350.367041,6.812905,12.203237,27.950732,97.738683,47.154988,20.292890,21.844982,25.377760,104.451237
5038,98.419265,42.664281,210.548841,84.557437,197.089334,10.388659,29.603303,142.710696,27.657525,7.703691,...,350.899223,6.668391,12.083881,28.263728,98.844811,47.497038,20.760968,21.005361,25.722090,105.481026
5039,100.233756,42.996281,211.623254,85.137454,201.071599,10.602619,30.195439,143.766224,28.125023,7.806366,...,360.178674,6.629824,11.910039,28.056946,99.887735,47.688886,20.499735,20.971418,26.050033,103.093167


In [18]:
larrys_mc = MCSimulation(
    portfolio_data = larrys_ticker,
    weights = [.2, .2, .2, .2, .2],
    num_simulation = 500,
    num_trading_days = 252*20)

In [19]:
# Run a Monte Carlo simulation to forecast five years cumulative returns with 60% AT&T stock
larrys_mc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.012154,0.998976,0.994183,1.001734,1.029663,1.019518,1.027188,0.997836,0.995032,0.986477,...,1.020761,0.983176,0.984920,1.008749,0.997895,0.999096,0.990579,1.006852,0.999382,0.993077
2,1.034676,0.996582,1.020463,0.999814,1.019998,1.037785,1.016770,0.975792,1.001149,1.006314,...,1.034529,0.957988,0.978850,1.002899,1.016669,1.010759,0.966253,0.993499,0.981748,0.953900
3,1.053378,1.003564,1.003282,1.004848,1.020881,1.013396,1.011178,0.968310,1.008584,1.025462,...,0.999966,0.964391,0.967693,1.001472,1.020461,1.016715,0.968222,0.990263,1.008043,0.937767
4,1.041271,0.996209,0.988039,0.994652,1.030425,1.002072,1.026162,0.965408,1.045208,1.031781,...,1.010938,0.946020,0.996051,0.989730,1.047666,1.007945,0.959112,0.990123,1.049550,0.931079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5036,216.180476,348.392658,241.719213,287.273898,262.000762,149.089386,440.262691,224.801596,440.015372,249.577439,...,556.345294,440.880251,461.264480,364.464472,2330.358550,650.673685,537.632594,413.989751,695.323460,100.041169
5037,219.852262,342.412871,245.115342,288.307413,266.313450,151.469864,442.135337,230.127043,429.064477,244.705153,...,547.816185,440.998829,462.199175,368.640083,2306.549373,645.198038,538.319713,406.198117,706.000609,101.780728
5038,213.216026,336.625073,247.611603,282.906758,278.786649,150.828703,442.633243,229.433489,424.987990,245.885210,...,542.034148,442.904914,462.648679,371.043640,2298.276404,643.735541,542.828755,398.877711,707.963257,101.282245
5039,212.272294,338.021986,250.898103,279.567249,274.153052,151.090782,434.412319,229.201106,414.752493,242.693845,...,552.429304,448.570940,473.806749,362.352370,2282.028068,650.915980,531.754801,390.824488,709.301918,99.448375


In [50]:
# Define Panel Visualization Functions
def daily_returns_plot():
    """Daily Returns"""
    
    # YOUR CODE HERE!
    combined_df = pd.concat([etf_df, joe_stock_df, joe_fid_df, larry_df], axis = 'columns', join = 'inner').dropna()
    daily_returns_plot = px.line(
    combined_df,
    x = combined_df.index,
    y= combined_df.columns[0:6],
    title = "Daily Returns Comparison")
    return daily_returns_plot
def cumulative_returns_plot():
    """Cumulative Returns"""
    
    # YOUR CODE HERE!
    combined_df = pd.concat([etf_df, joe_stock_df, joe_fid_df, larry_df], axis = 'columns', join = 'inner').dropna()
    combined_cumulative = (1 + combined_df).cumprod()
    cumulative_returns_plot = px.line(
    combined_cumulative,
    x = combined_cumulative.index,
    y = combined_cumulative.columns[0:6],
    title = "Cumulative Returns Comparison")
    return cumulative_returns_plot
def sharpe_ratio_plot():
    """Sharpe Ratio"""
    # YOUR CODE HERE!
    all_portfolio_std = sharpe_df.std()
    sharpe_ratios = pd.DataFrame((sharpe_df.mean() * 252) / (all_portfolio_std * np.sqrt(252)))
    sharpe_ratio_plot = px.bar(
    sharpe_ratios,
    x = sharpe_ratios.index,
    y = sharpe_ratios.columns[0:12],
    title = "Sharpe Ratio Contrast"
    )
    return sharpe_ratio_plot

In [51]:
# Create panels to structure the layout of the dashboard
daily_returns_columns = pn.Column("## Daily Returns Plots for All Portfolios", daily_returns_plot())
cumulative_column = pn.Column("## Cumulative Returns of All Portfolios", cumulative_returns_plot())
sharpe_column = pn.Column("## Sharpe Ratio Comparison", sharpe_ratio_plot())
# Create a Title for the Dashboard
# YOUR CODE HERE!
Portfolio_Analysis_dashboard = '# How does the average Joe or Larry stack up to an ETF'

# Create a tab layout for the dashboard
# YOUR CODE HERE!
Portfolio_Analysis_dashboard = pn.Tabs(
    ("Daily Returns", daily_returns_columns),
    ("Cumulative Returns", cumulative_column),
    ("Sharpe Ratio", sharpe_column)
    
)
# Serve the# dashboard
# YOUR CODE HERE!
Portfolio_Analysis_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)